In [ ]:
import json
import sys
from sklearn.model_selection import train_test_split
from typing import List, Tuple, Set
import networkx as nx
from tqdm import tqdm
import matplotlib.pyplot as plt
from collections import namedtuple
import copy
from networkx.drawing.nx_agraph import write_dot
from networkx.algorithms.community import greedy_modularity_communities
from networkx.drawing import nx_pydot
from sexpdata import loads
%matplotlib inline

In [ ]:
def read_and_parse_hrg(pth: str):
    hrg = []
    with open(pth, "r") as f:
        for line in f:
            hrg.append(line.strip())
    hrg = " ".join(hrg)
    parsed_hrg = loads(hrg)
    return parsed_hrg

In [ ]:
def plot_hrg(pth):
    parsed_hrg = read_and_parse_hrg(pth)
    for root in parsed_hrg:
        G = make_tree(root)
        plot_graph(G)

def make_tree(root, G=None):
    if G is None:
        G = nx.DiGraph()
    G.add_node(parse_avl(root))
    for daughter in root[1:]:
        make_tree(daughter, G)
        G.add_edge(parse_avl(root), parse_avl(daughter))
    return G

def parse_avl(node):
    return node[0][1][1]

def plot_graph(G):
    nx.draw(G, arrows=True, with_labels=True, pos=nx.nx_agraph.graphviz_layout(G, prog='dot'),
           node_size=1800, node_color="skyblue")
    plt.show()

In [ ]:
plot_hrg("../data/arctic_a0001.hrg")

In [ ]:
def to_nx_with_attr(pth) -> nx.Graph:
        parsed_hrg = read_and_parse_hrg(pth)
        G = nx.DiGraph()
        edge_attr = {}
        for i in range(len(parsed_hrg)):  # for each word
            word = parsed_hrg[i][0][1][1]
            G.add_node(word)
            num_syllables = len(parsed_hrg[i][1:])
            for j in range(1, num_syllables + 1):  # for each syl
                num_components = len(parsed_hrg[i][j]) - 1 
                for k in range(1, num_components + 1):
                    
                    syl = f"{parsed_hrg[i][j][k][0][1][1]}-{i}"  # each syl is tied to a word
                    syl_dur = str(round(parsed_hrg[i][j][k][0][2][1] * 100, 2))
                    syl_end = str(round(parsed_hrg[i][j][k][0][3][1] * 100, 2))
                    G.add_node(syl)
                    G.add_node(syl_end)
                    G.add_node(syl_dur)
                    
                    G.add_edge(word, syl)
                    edge_attr[(word, syl)] = "syl"
                    

                    G.add_edge(syl, syl_end)
                    edge_attr[(syl, syl_end)] = "end"
                    
                    G.add_edge(syl, syl_dur)
                    edge_attr[(syl, syl_dur)] = "dur"                   
                    
        return G, edge_attr

In [ ]:
def attr_plot(G, edge_attr):
    
    f = plt.figure(figsize=(25, 25))

    nx.set_edge_attributes = edge_attr
    layout = nx.nx_agraph.graphviz_layout(G, prog='neato')
    nx.draw_networkx_edge_labels(G, edge_labels=edge_attr, font_color='red', pos = layout)


    nx.draw(G, arrows=True, with_labels=True, edge_attributes = nx.get_edge_attributes(G,'dot'),
            pos = layout, node_size=1800, node_color="pink")

In [ ]:
G, edge_attr = to_nx_with_attr("../data/arctic_a0001.hrg")

In [ ]:
attr_plot(G, edge_attr)